<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
</center>



## <center>Tema: Predicción de Fallas en Camiones de Minería</center>

### <center>Practico 2 - Curaci&oacute;n</center>

### Mentor:
* Grando, Lucas

###  Integrantes:
* Fabro, Juan
* Moreno, Natalia

<h1> Práctico 2 - Curación</h1>
<h3> Análisis Exploratorio y Curación de Datos - 2019 </h3>

Trabajaremos con un dataset que contiene muestras de aceite extraidas a camiones de minería. Estas muestras son extraíadas a diferentes componentes de los camiones (Motor, Diferencial, etc)

Las muestras se envían a un laboratorio que analiza los diferentes componentes químicos de las mismas (Hierro, Cromo, presencia de Agua, horas de utitización, etc), luego de analizarlas emiten un informe indicando el estado de la muesta,  lo que permite a los analistas encontar posibles fallas en los equipos, previmiendo futuras roturas.

El dataset cuenta con X features, siendo las más importantes

* Componente: Indica a que componente pertenece la muestra
* Horas Funcionamiento: Indica la cantidad de horas de funcionamiento del camión (sería como el kilometraje de los camiones)
* Horas del Aceite: Representa la cantidad de horas de utilización del aceite (este dato es importante dado que a medida que, a mayor horas de uso del aceite, el mismo comienza a desgastarse)
* Resultado: (El laboratorio indica si la muestra de aceite está Bien = 1, Regular=2, Mal=3)
* St: Presencia de Hollin en el Aceite
* Al:  Presencia de Aluminio en el Aceite
* Fe	 Presencia de Hierro en el Aceite
* Si  Presencia de Silicio en el Aceite
* Na	 Presencia de Sodio en el Aceite
* Visco: Viscosidad del aceite

NOTA: se modifica le dataset y se incluyen el feature
* Fecha de Análisis: Indica cuando fué analizada la muestra por el laboratorio
    

1. Comenzamos realizando el setup del ambiente de trabajo

In [104]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

In [105]:
# Cargamos el set de datos 
#(para ejecutar esta linea, tal cual esta, el archivo csv debe enccontrarse en el mismo directorio que la notebook)

# Importamos solo las rows con datos para evitar operaciones extras con los falsos NaN dados por la ultima fila vacia
dataset = pd.read_csv('OilDataSetCuracion.csv', nrows=21432)

In [106]:
# Configuramos una semilla para reproducibilidad. Presentamos una muestra de 5 registros
np.random.seed(0)
dataset.sample(5)

,Equipo,Componente,Id,Resultado,Horas Funcionamiento,Horas del Aceite,Fecha de Análisis,B,Nit,Oxi,...,Zn,Ag,Ti,V40,V100,TBN,TAN,ISO14,ISO4,ISO6
2054,1368,Masa Derecha,281942,1,48434,407.0,2019-03-27 00:00:00.000,115.0,NaN,NaN,...,21.0,0.0,0.0,NaN,23.6,NaN,NaN,13.0,21.0,18.0
12132,1407,Motor,201075,1,36796,256.0,2018-05-07 00:00:00.000,109.0,6.0,16.0,...,1243.0,0.0,0.0,NaN,14.4,9.3,NaN,NaN,NaN,NaN
208,1359,Masa Izquierda,279965,1,49236,296.0,2019-03-21 00:00:00.000,84.3,NaN,NaN,...,79.2,0.2,0.1,22.7,NaN,NaN,NaN,NaN,NaN,NaN
19728,1424,Masa Izquierda,276168,1,35580,535.0,2019-03-01 00:00:00.000,140.0,NaN,NaN,...,25.0,0.0,0.0,NaN,25.0,NaN,NaN,13.0,22.0,19.0
18240,1431,Transmisión,260135,1,13595,1180.0,2018-12-27 00:00:00.000,0.0,NaN,NaN,...,1139.0,0.0,0.0,NaN,10.7,NaN,NaN,14.0,22.0,17.0


2. Vamos a observar el tipo de dato de cada feature del dataset para conocer si detecta correctamente los tipos de datos:

In [107]:
dataset.dtypes

Equipo                    int64
Componente               object
Id                        int64
Resultado                 int64
Horas Funcionamiento      int64
Horas del Aceite        float64
Fecha de Análisis        object
B                       float64
Nit                     float64
Oxi                     float64
Sul                     float64
St                      float64
V                       float64
Al                      float64
Cr                      float64
Cu                      float64
Fe                      float64
Pb                      float64
Mo                      float64
Ni                      float64
Sn                      float64
Si                      float64
K                       float64
Na                      float64
W                        object
F                        object
A                        object
ISO                      object
PQI                     float64
Ba                      float64
Ca                      float64
Mg      

Algunas columnas son interpretadas con un tipo de datos diferentes al valor que realmente contienen los datos. 
Vamos a corregirlos:

In [108]:
# Se actualiza la columna Fecha de Análisis a tipo fecha, en lugar de objeto
dataset = pd.read_csv('OilDataSetCuracion.csv', parse_dates=["Fecha de Análisis"])
dataset.dtypes

Equipo                          object
Componente                      object
Id                             float64
Resultado                      float64
Horas Funcionamiento           float64
Horas del Aceite               float64
Fecha de Análisis       datetime64[ns]
B                              float64
Nit                            float64
Oxi                            float64
Sul                            float64
St                             float64
V                              float64
Al                             float64
Cr                             float64
Cu                             float64
Fe                             float64
Pb                             float64
Mo                             float64
Ni                             float64
Sn                             float64
Si                             float64
K                              float64
Na                             float64
W                               object
F                        

In [109]:
# Se actualizan las columnas Resultado, ISO14, ISO6 e ISO4 a datos de tipo enteros

#dataset['Resultado'] = pd.to_numeric(dataset['Resultado'], downcast='integer')
dataset['Resultado'] = dataset['Resultado'].astype('int64', errors='ignore')
dataset['ISO14'] = dataset['Resultado'].astype('int64', errors='ignore')
dataset['ISO6'] = dataset['Resultado'].astype('int64', errors='ignore')
dataset['ISO4'] = dataset['Resultado'].astype('int64', errors='ignore')

In [110]:
dataset.dtypes

Equipo                          object
Componente                      object
Id                             float64
Resultado                      float64
Horas Funcionamiento           float64
Horas del Aceite               float64
Fecha de Análisis       datetime64[ns]
B                              float64
Nit                            float64
Oxi                            float64
Sul                            float64
St                             float64
V                              float64
Al                             float64
Cr                             float64
Cu                             float64
Fe                             float64
Pb                             float64
Mo                             float64
Ni                             float64
Sn                             float64
Si                             float64
K                              float64
Na                             float64
W                               object
F                        

3. Vamos a verificar entre que fechas hay muestras de aceites:

In [111]:
dataset['Fecha de Análisis'].min()

Timestamp('2017-10-29 00:00:00')

In [112]:
dataset['Fecha de Análisis'].max()

Timestamp('2019-06-03 00:00:00')

4. Verificamos si existen registros duplicados en el dataset:

In [113]:
dataset[dataset.duplicated(keep=False)]

,Equipo,Componente,Id,Resultado,Horas Funcionamiento,Horas del Aceite,Fecha de Análisis,B,Nit,Oxi,...,Zn,Ag,Ti,V40,V100,TBN,TAN,ISO14,ISO4,ISO6


Aparentemente no hay registros duplicados. Vamos a controlar si tomando el feature Id como &iacute;ndice sigue sin haber registros repetidos.

In [114]:
dataset_Id = pd.read_csv('OilDataSetCuracion.csv', parse_dates=["Fecha de Análisis"], index_col=['Id'])

In [115]:
repetidos = dataset_Id[dataset_Id.duplicated(keep=False)]
repetidos.shape

(239, 43)

Para el caso en que tomemos el feature Id como indice del dataset encontramos 239 filas repetidas.

Vemos algunos de los registros repetidos

In [116]:
repetidos.sort_values(by=['Equipo', 'Componente', 'Resultado', 'Horas Funcionamiento']).head()

,Equipo,Componente,Resultado,Horas Funcionamiento,Horas del Aceite,Fecha de Análisis,B,Nit,Oxi,Sul,...,Zn,Ag,Ti,V40,V100,TBN,TAN,ISO14,ISO4,ISO6
Id,,,,,,,,,,,,,,,,,,,,,
254898.0,1359,Convertidor,1.0,46874.0,886.0,2018-10-20,1.1,NaN,NaN,NaN,...,1152.0,0.1,0.2,10.73,NaN,NaN,NaN,NaN,NaN,NaN
254897.0,1359,Convertidor,1.0,46874.0,886.0,2018-10-20,1.1,NaN,NaN,NaN,...,1152.0,0.1,0.2,10.73,NaN,NaN,NaN,NaN,NaN,NaN
254907.0,1359,Diferencial Trasero,3.0,46874.0,2578.0,2018-10-20,84.3,NaN,NaN,NaN,...,54.2,0.1,0.1,24.02,NaN,NaN,NaN,NaN,NaN,NaN
254908.0,1359,Diferencial Trasero,3.0,46874.0,2578.0,2018-10-20,84.3,NaN,NaN,NaN,...,54.2,0.1,0.1,24.02,NaN,NaN,NaN,NaN,NaN,NaN
254912.0,1359,Mando Final TD,3.0,46874.0,2578.0,2018-10-20,80.2,NaN,NaN,NaN,...,42.8,0.1,0.1,24.02,NaN,NaN,NaN,NaN,NaN,NaN


5. Despersonalización de datos

Vamos a aplicar dos técnicas, en esta primera instancia aplicamos un hash o md5:

In [117]:
from hashlib import md5

In [118]:
# Function thar convert val to hash
def hashit(val):
    if isinstance(val, float): 
        return str(val)
    return md5(val.encode('utf-8')).hexdigest()

In [119]:
# Agregamos una nueva columna con el Id convertido a md5, de manera de conservar Id para el próximo calculo
dataset['HId'] = dataset['Id'].apply(hashit)
dataset.sample(5)

,Equipo,Componente,Id,Resultado,Horas Funcionamiento,Horas del Aceite,Fecha de Análisis,B,Nit,Oxi,...,Ag,Ti,V40,V100,TBN,TAN,ISO14,ISO4,ISO6,HId
12582,1412,Masa Izquierda,212643.0,1.0,34610.0,589.0,2018-06-21,110.0,NaN,NaN,...,0.0,0.0,NaN,24.5,NaN,NaN,1.0,1.0,1.0,212643.0
20830,1429,Sist. Hidráulico,188661.0,1.0,27143.0,1597.0,2018-03-18,1.0,NaN,NaN,...,0.0,0.0,NaN,6.4,NaN,NaN,1.0,1.0,1.0,188661.0
3409,1375,Diferencial Trasero,292421.0,1.0,50528.0,985.0,2019-05-03,86.0,NaN,16.0,...,0.1,0.1,23.35,NaN,NaN,1.3,1.0,1.0,1.0,292421.0
7752,1389,Motor,258601.0,1.0,46516.0,295.0,2018-12-20,45.0,8.0,18.0,...,0.0,0.0,NaN,15.5,9.9,NaN,1.0,1.0,1.0,258601.0
7538,1390,Diferencial Trasero,284794.0,1.0,49787.0,320.0,2019-04-07,93.0,NaN,NaN,...,0.0,0.0,NaN,24.0,NaN,NaN,1.0,1.0,1.0,284794.0


Tambien podriamos usar algun tipo de codificacion como lo es base64

In [120]:
import base64

In [121]:
# Funcion codificadora base64 to ascii
def encondings(val):
    bs64 = base64.b64encode(bytes(str(val).encode('utf-8')))
    return bs64.decode('ascii')

In [122]:
# Agregamos una columna con el Id en base64
dataset['EId'] = dataset['Id'].apply(encondings)
dataset.sample(5)

,Equipo,Componente,Id,Resultado,Horas Funcionamiento,Horas del Aceite,Fecha de Análisis,B,Nit,Oxi,...,Ti,V40,V100,TBN,TAN,ISO14,ISO4,ISO6,HId,EId
18933,1834,Convertidor,193636.0,2.0,12195.0,240.0,2018-04-08,0.0,NaN,NaN,...,0.0,NaN,11.2,NaN,NaN,2.0,2.0,2.0,193636.0,MTkzNjM2LjA=
13816,1419,Masa Izquierda,286732.0,1.0,36722.0,691.0,2019-04-13,83.3,NaN,NaN,...,0.6,22.91,NaN,NaN,NaN,1.0,1.0,1.0,286732.0,Mjg2NzMyLjA=
546,1361,Masa Derecha,279976.0,3.0,50022.0,326.0,2019-03-21,86.9,NaN,NaN,...,0.1,22.58,NaN,NaN,NaN,3.0,3.0,3.0,279976.0,Mjc5OTc2LjA=
18803,1421,Diferencial Trasero,266588.0,1.0,33728.0,962.0,2019-01-21,79.8,NaN,NaN,...,0.1,23.95,NaN,NaN,NaN,1.0,1.0,1.0,266588.0,MjY2NTg4LjA=
3300,1376,Masa Izquierda,209452.0,1.0,43838.0,NaN,2018-05-29,104.0,NaN,NaN,...,0.0,NaN,24.6,NaN,NaN,1.0,1.0,1.0,209452.0,MjA5NDUyLjA=


6. Visualizamos las etiquetas de las features para reemplazar los caracteres no ascii:

In [123]:
dataset.columns

Index(['Equipo', 'Componente', 'Id', 'Resultado', 'Horas Funcionamiento',
       'Horas del Aceite', 'Fecha de Análisis', 'B', 'Nit', 'Oxi', 'Sul', 'St',
       'V', 'Al', 'Cr', 'Cu', 'Fe', 'Pb', 'Mo', 'Ni', 'Sn', 'Si', 'K', 'Na',
       'W', 'F', 'A', 'ISO', 'PQI', 'Ba', 'Ca', 'Mg', 'Mn', 'P', 'Zn', 'Ag',
       'Ti', 'V40', 'V100', 'TBN', 'TAN', 'ISO14', 'ISO4', 'ISO6', 'HId',
       'EId'],
      dtype='object')

Observamos que los nombres de las columnas tienen espacios, vamos a eliminarlos para una mejor manipulación de los datos:

In [124]:
dataset.columns = dataset.columns.str.replace(' ', '_')
dataset.head()

,Equipo,Componente,Id,Resultado,Horas_Funcionamiento,Horas_del_Aceite,Fecha_de_Análisis,B,Nit,Oxi,...,Ti,V40,V100,TBN,TAN,ISO14,ISO4,ISO6,HId,EId
0,1355,Mando Final TD,273615.0,1.0,21950.0,409.0,2019-02-20,96.0,4.0,8.0,...,0.0,NaN,24.5,NaN,NaN,1.0,1.0,1.0,273615.0,MjczNjE1LjA=
1,1355,Mando Final TI,294763.0,3.0,23153.0,424.0,2019-05-12,84.0,3.0,7.0,...,0.0,NaN,24.2,NaN,NaN,3.0,3.0,3.0,294763.0,Mjk0NzYzLjA=
2,1355,Masa Derecha,278700.0,3.0,22349.0,399.0,2019-03-14,114.0,NaN,NaN,...,1.0,NaN,25.0,NaN,NaN,3.0,3.0,3.0,278700.0,Mjc4NzAwLjA=
3,1355,Motor,273586.0,3.0,21950.0,409.0,2019-02-20,51.0,9.0,22.0,...,0.0,NaN,13.7,9.1,NaN,3.0,3.0,3.0,273586.0,MjczNTg2LjA=
4,1355,Sist de Dirección,278549.0,1.0,22349.0,808.0,2019-03-14,2.0,NaN,NaN,...,0.0,NaN,10.8,NaN,NaN,1.0,1.0,1.0,278549.0,Mjc4NTQ5LjA=


Reemplazamos también los acentos:

In [125]:
dataset = dataset.rename(columns={'Fecha_de_Análisis': 'Fecha_de_Analisis'})
dataset.columns

Index(['Equipo', 'Componente', 'Id', 'Resultado', 'Horas_Funcionamiento',
       'Horas_del_Aceite', 'Fecha_de_Analisis', 'B', 'Nit', 'Oxi', 'Sul', 'St',
       'V', 'Al', 'Cr', 'Cu', 'Fe', 'Pb', 'Mo', 'Ni', 'Sn', 'Si', 'K', 'Na',
       'W', 'F', 'A', 'ISO', 'PQI', 'Ba', 'Ca', 'Mg', 'Mn', 'P', 'Zn', 'Ag',
       'Ti', 'V40', 'V100', 'TBN', 'TAN', 'ISO14', 'ISO4', 'ISO6', 'HId',
       'EId'],
      dtype='object')

7. Vamos a revisar la cantidad de datos faltantes en las features Fe y Na

In [126]:
# Observamos 740 datos faltantes para el hierro
nan_fe = dataset[dataset['Fe'].isnull()]['Fe']
nan_fe.shape

(740,)

In [127]:
len(dataset.dropna(subset=["Fe"]))/len(dataset)

0.9654738020809033

Como podemos ver el 96,55% de las filas tienen el feature Fe no nulo

In [128]:
# Observamos 743 datos faltantes para el sodio
nan_na = dataset[dataset['Na'].isnull()]['Na']
nan_na.shape

(743,)

In [129]:
len(dataset.dropna(subset=["Na"]))/len(dataset)

0.9653338310082583

Y el 96,53% de las filas tienen el feature Na no nulo

Completamos los datos faltantes con la media:

In [130]:
dataset['Fe'].fillna((dataset['Fe'].mean()), inplace=True)

In [131]:
# Observamos que ya no hay datos faltantes para el hierro
nan_fe = dataset[dataset['Fe'].isnull()]['Fe']
nan_fe.shape

(0,)

In [132]:
dataset['Na'].fillna((dataset['Na'].mean()), inplace=True)

In [133]:
# Observamos que ya no hay datos faltantes para el sodio
nan_na = dataset[dataset['Na'].isnull()]['Na']
nan_na.shape

(0,)